### Training with a better optimizer

The original model is trained using stochastic gradient descent on a single sample at a time, with a fixed learning rate.

To see if a more sophisticated optimizer will help, we will try to train the same model as in the previous notebook, but with a version of the Adam optimizer. Adam is a combination of two gradient descent "variations" we have discussed previously:

* Momentum - uses a velocity vector that accumulates gradient of past steps.
* RMSProp - normalizes parameter update using per-parameter EWMA of square of gradient.

and it is often used as a starting point when training deep neural networks. 

We are also going to "decay" the learning rate as we iterate over the number of epochs.

And, we will also double the number of training epochs from 23 to 46.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.set_num_threads(2) # for performance

In [ ]:

class ModernLossNet(nn.Module):

    def __init__(self):
        super().__init__()

        # initialization as described in the paper to my best ability, but it doesn't look right...
        winit = lambda fan_in, *shape: (torch.rand(*shape) - 0.5) * 2 * 2.4 / fan_in**0.5
        macs = 0 # keep track of MACs (multiply accumulates)
        acts = 0 # keep track of number of activations

        # H1 layer parameters and their initialization
        self.H1w = nn.Parameter(winit(5*5*1, 12, 1, 5, 5))
        self.H1b = nn.Parameter(torch.zeros(12, 8, 8)) # presumably init to zero for biases
        assert self.H1w.nelement() + self.H1b.nelement() == 1068
        macs += (5*5*1) * (8*8) * 12
        acts += (8*8) * 12

        # H2 layer parameters and their initialization
        """
        H2 neurons all connect to only 8 of the 12 input planes, with an unspecified pattern
        I am going to assume the most sensible block pattern where 4 planes at a time connect
        to differently overlapping groups of 8/12 input planes. We will implement this with 3
        separate convolutions that we concatenate the results of.
        """
        self.H2w = nn.Parameter(winit(5*5*8, 12, 8, 5, 5))
        self.H2b = nn.Parameter(torch.zeros(12, 4, 4)) # presumably init to zero for biases
        assert self.H2w.nelement() + self.H2b.nelement() == 2592
        macs += (5*5*8) * (4*4) * 12
        acts += (4*4) * 12

        # H3 is a fully connected layer
        self.H3w = nn.Parameter(winit(4*4*12, 4*4*12, 30))
        self.H3b = nn.Parameter(torch.zeros(30))
        assert self.H3w.nelement() + self.H3b.nelement() == 5790
        macs += (4*4*12) * 30
        acts += 30

        # output layer is also fully connected layer
        self.outw = nn.Parameter(winit(30, 30, 10))
        self.outb = nn.Parameter(-torch.ones(10)) # 9/10 targets are -1, so makes sense to init slightly towards it
        assert self.outw.nelement() + self.outb.nelement() == 310
        macs += 30 * 10
        acts += 10

        self.macs = macs
        self.acts = acts

    def forward(self, x):

        # x has shape (1, 1, 16, 16)
        x = F.pad(x, (2, 2, 2, 2), 'constant', -1.0) # pad by two using constant -1 for background
        x = F.conv2d(x, self.H1w, stride=2) + self.H1b
        x = torch.tanh(x)

        # x is now shape (1, 12, 8, 8)
        x = F.pad(x, (2, 2, 2, 2), 'constant', -1.0) # pad by two using constant -1 for background
        slice1 = F.conv2d(x[:, 0:8], self.H2w[0:4], stride=2) # first 4 planes look at first 8 input planes
        slice2 = F.conv2d(x[:, 4:12], self.H2w[4:8], stride=2) # next 4 planes look at last 8 input planes
        slice3 = F.conv2d(torch.cat((x[:, 0:4], x[:, 8:12]), dim=1), self.H2w[8:12], stride=2) # last 4 planes are cross
        x = torch.cat((slice1, slice2, slice3), dim=1) + self.H2b
        x = torch.tanh(x)

        # x is now shape (1, 12, 4, 4)
        x = x.flatten(start_dim=1) # (1, 12*4*4)
        x = x @ self.H3w + self.H3b
        x = torch.tanh(x)

        # x is now shape (1, 30)
        x = x @ self.outw + self.outb
        # Note: we deleted the tanh activation here!

         # x is finally shape (1, 10)
        return x

In [ ]:
# The following code is adapted from "modern.py" in the repository.

# Note: different initial learning rate
learning_rate = 3e-4

# init rng
torch.manual_seed(1337)
np.random.seed(1337)
torch.use_deterministic_algorithms(True)

# init a model
model = ModernLossNet()
print("model stats:")
print("# params:      ", sum(p.numel() for p in model.parameters())) # in paper total is 9,760
print("# MACs:        ", model.macs)
print("# activations: ", model.acts)

# init data
Xtr, Ytr = torch.load('train1989.pt')
Xte, Yte = torch.load('test1989.pt')

# init optimizer
# Note: use the the AdamW optimizer
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

def eval_split(split):
    # eval the full train/test set, batched implementation for efficiency
    model.eval()
    X, Y = (Xtr, Ytr) if split == 'train' else (Xte, Yte)
    Yhat = model(X)
    loss = F.cross_entropy(Yhat, Y.argmax(dim=1))
    err = torch.mean((Y.argmax(dim=1) != Yhat.argmax(dim=1)).float())
    print(f"eval: split {split:5s}. loss {loss.item():e}. error {err.item()*100:.2f}%. misses: {int(err.item()*Y.size(0))}")
  
# Note: increase number of training passes
for pass_num in range(46):

    # Note: also implement learning rate decay
    alpha = pass_num / 45
    for g in optimizer.param_groups:
        g['lr'] = (1 - alpha) * learning_rate + alpha * (learning_rate / 3)

    # perform one epoch of training
    model.train()
    for step_num in range(Xtr.size(0)):

        # fetch a single example into a batch of 1
        x, y = Xtr[[step_num]], Ytr[[step_num]]

        # forward the model and the loss
        yhat = model(x)
        loss = F.cross_entropy(yhat, y.argmax(dim=1))

        # calculate the gradient and update the parameters
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

    # after epoch epoch evaluate the train and test error / metrics
    print(pass_num + 1)
    eval_split('train')
    eval_split('test')

# save final model to file
torch.save(model.state_dict(), 'adam_model.pt')

model stats:
# params:       9760
# MACs:         63660
# activations:  1000
1
eval: split train. loss 3.553850e-01. error 10.08%. misses: 734
eval: split test . loss 3.418252e-01. error 10.31%. misses: 206
2
eval: split train. loss 2.473212e-01. error 7.20%. misses: 524
eval: split test . loss 2.514379e-01. error 7.22%. misses: 144
3
eval: split train. loss 1.973256e-01. error 5.58%. misses: 406
eval: split test . loss 2.185514e-01. error 6.13%. misses: 122
4
eval: split train. loss 1.690831e-01. error 4.77%. misses: 347
eval: split test . loss 2.048475e-01. error 5.73%. misses: 115
5
eval: split train. loss 1.465219e-01. error 4.32%. misses: 314
eval: split test . loss 1.939704e-01. error 5.28%. misses: 105
6
eval: split train. loss 1.272241e-01. error 3.72%. misses: 271
eval: split test . loss 1.844373e-01. error 5.18%. misses: 104
7
eval: split train. loss 1.110661e-01. error 3.13%. misses: 228
eval: split test . loss 1.769724e-01. error 4.98%. misses: 99
8
eval: split train. loss 

We seem to have improved the test performance - it is now below 4% error. We are still achieving 0 error on the training set, so we remain concerned about potential overfitting.